In [14]:
# Data Extraction
import pandas as pd 
import numpy as np
factors = pd.read_excel('./midterm_2_data.xlsx', sheet_name = 'factors (excess returns)').set_index('Date')
portfolio = pd.read_excel('./midterm_2_data.xlsx', sheet_name = 'portfolios (excess returns)').set_index('Date')

annualization_factor = 12

import statsmodels.api as sm


def time_series_regression(portfolio, factors, FF3F = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] 
        ff_report.loc[portf, 'Market beta'] = res.params[1]
        if FF3F:
            ff_report.loc[portf, 'Value beta'] = res.params[2] 
            ff_report.loc[portf, 'Momentum beta'] = res.params[3]
            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
    if resid:
        return bm_residuals
        
    return ff_report

In [15]:
ts_FF = time_series_regression(portfolio, factors,True)
ts_FF['alpha_hat']*=12
ts_FF[['alpha_hat','Market beta', 'Value beta','Momentum beta','R-squared']]

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'Market beta'] = res.params[1]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'Value beta'] = res.params[2]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To

,alpha_hat,Market beta,Value beta,Momentum beta,R-squared
NoDur,0.029253,0.739522,0.204580,0.049333,0.617919
Durbl,0.010734,1.271865,0.173595,-0.320023,0.613493
Manuf,-0.000996,1.049482,0.197462,-0.036704,0.870268
Enrgy,-0.015117,0.992222,0.637006,0.075170,0.465602
HiTec,0.028207,1.154959,-0.637135,-0.140638,0.829498
Telcm,0.003506,0.837326,0.094363,-0.084518,0.588052
Shops,0.026739,0.946928,-0.042222,-0.015005,0.742161
Hlth,0.031862,0.757605,-0.119928,0.074058,0.580514
Utils,0.013710,0.527879,0.353033,0.108622,0.342654
Other,-0.019780,1.115433,0.426753,-0.048678,0.910098


In [16]:
ts_FF.head()

,alpha_hat,Market beta,Value beta,Momentum beta,info_ratio,treynor_ratio,R-squared,Tracking Error
NoDur,0.029253,0.739522,0.204580,0.049333,0.331568,0.134449,0.617919,0.088227
Durbl,0.010734,1.271865,0.173595,-0.320023,0.063625,0.080186,0.613493,0.168711
Manuf,-0.000996,1.049482,0.197462,-0.036704,-0.015822,0.085455,0.870268,0.062979
Enrgy,-0.015117,0.992222,0.637006,0.075170,-0.091485,0.089360,0.465602,0.165238
HiTec,0.028207,1.154959,-0.637135,-0.140638,0.294943,0.086957,0.829498,0.095634


In [18]:
mean_portfolios = (portfolio.mean()*12).to_frame('Mean Portfolio excess returns')
CS = time_series_regression(mean_portfolios, ts_FF.loc[:,['Market beta', 'Value beta','Momentum beta']], True)
CS.columns = ['Annualized Intercept','Market regression coefficient','Value regression coefficients',
            'Momentum regression coefficients','info_ratio','treynor_ratio','R-squared','Tracking Error']


/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'Market beta'] = res.params[1]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'Value beta'] = res.params[2]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To

In [19]:
CS

,Annualized Intercept,Market regression coefficient,Value regression coefficients,Momentum regression coefficients,info_ratio,treynor_ratio,R-squared,Tracking Error
Mean Portfolio excess returns,0.063716,0.031992,-0.015767,0.030301,23.081584,34.025053,0.366198,0.033126


In [22]:
# annualized factor premia 

# this is for the time series regression
factors.mean() * 12 

MKT    0.083853
HML    0.025028
UMD    0.061692
dtype: float64

In [23]:
# this is for the cross sectional regression 
CS[['Market regression coefficient','Value regression coefficients',
    'Momentum regression coefficients']]

,Market regression coefficient,Value regression coefficients,Momentum regression coefficients
Mean Portfolio excess returns,0.031992,-0.015767,0.030301


In [24]:
resid = time_series_regression(mean_portfolios, ts_FF.loc[:,['Market beta', 'Value beta','Momentum beta']], False, True)
mae = pd.DataFrame([(((ts_FF['alpha_hat']).abs().mean()))], columns = ['MAE (%)'], index = ['TS'])
mae.loc['CS'] = abs(resid).mean()[0]
mae

/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'Market beta'] = res.params[1]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/1113374628.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
/var/folders/qq/h84pzsw93xbd60q11ssc5v640000gn/T/ipykernel_6833/3278838785.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consis

,MAE (%)
TS,0.017990
CS,0.007915
